In [ ]:
import numpy as np

_frequencies = [1265, 897, 643, 1190, 521, 1688, 778, 1999, 1690, 1433, 1796, 1266, 1046, 1353]
frequencies = np.array(_frequencies) / np.array(_frequencies).sum()

frequencies

In [ ]:
a = TiDBHypo("imdbload")
# b = a.execute_create_hypo("aka_name#surname_pcode")
b = a.get_hypo_indexes_from_one_table("aka_name")
b
# a.test()


In [ ]:

string = "KEY `hypo_aka_name_name_pcode_nf_idx` (`name_pcode_nf`) /* HYPO INDEX */"

if "HYPO INDEX" in string:
    tmp = string.split("`")
    idx_name = tmp[1]
    table_name = "aka_name"
    hypo = f"{table_name}.{idx_name}"
    print(hypo)
    
    

In [ ]:
import numpy as np

np.zeros(60, dtype=np.cfloat)

In [ ]:
import pymysql


def get_hypo_indexes_from_table(conn, table_name):
    statement = f"show create table {table_name}"
    cur = conn.cursor()
    result = cur.execute(statement)
    hypo_indexes = []
    for line in result[1].split("\n"):
        if "HYPO INDEX" in line:
            tmp = line.split("`")
            hypo_indexes.append(tmp[1])
    return hypo_indexes
    
    

conn = pymysql.connect(host='127.0.0.1',
                     port=4000,
                     user='root',
                     password='',
                     database="imdbload",
                     local_infile=True)

cur = conn.cursor()



statement = "create index hypo_aka_name_name_pcode_nf_idx type hypo on aka_name (name_pcode_nf);"
s2 = "show create table aka_name;"


cur.execute(statement)

print(get_hypo_indexes_from_table(conn, "aka_name"))

cur.execute(s2)

print(cur.fetchall())

In [ ]:
indexes = [1.0, 0.0, 1.0]
index_candidates=["123","423123", "23123"]

selected_indexes = [index_candidates[i] for i, idx in enumerate(indexes) if idx == 1.0]

selected_indexes

In [ ]:
import numpy as np

action = np.random.randint(0, 65)



a = np.float64(0)
a

In [ ]:
l = ['customer#c_custkey,c_mktsegment', 'lineitem#l_commitdate,l_receiptdate', 'lineitem#l_orderkey,l_receiptdate', 'lineitem#l_orderkey,l_shipdate', 'lineitem#l_partkey', 'lineitem#l_partkey,l_quantity,l_extendedprice', 'lineitem#l_partkey,l_shipdate', 'lineitem#l_partkey,l_suppkey', 'lineitem#l_partkey,l_suppkey,l_shipdate', 'lineitem#l_receiptdate', 'lineitem#l_returnflag', 'lineitem#l_shipdate', 'lineitem#l_shipmode', 'lineitem#l_suppkey', 'lineitem#l_suppkey,l_partkey', 'lineitem#l_suppkey,l_partkey,l_shipdate', 'nation#n_name', 'nation#n_name,n_nationkey', 'nation#n_nationkey', 'nation#n_nationkey,n_name', 'nation#n_regionkey', 'orders#o_custkey', 'orders#o_custkey,o_orderdate', 'orders#o_orderkey', 'orders#o_orderkey,o_orderdate', 'orders#o_orderkey,o_orderdate,o_custkey', 'orders#o_orderkey,o_orderstatus', 'orders#o_orderstatus,o_orderkey', 'part#p_brand', 'part#p_brand,p_container,p_partkey', 'part#p_brand,p_partkey', 'part#p_brand,p_partkey,p_container', 'part#p_container,p_brand', 'part#p_container,p_brand,p_partkey', 'part#p_container,p_partkey', 'part#p_container,p_partkey,p_brand', 'part#p_partkey,p_brand,p_container', 'part#p_partkey,p_container', 'part#p_partkey,p_container,p_brand', 'part#p_partkey,p_type', 'partsupp#ps_availqty', 'partsupp#ps_partkey', 'partsupp#ps_partkey,ps_availqty,ps_suppkey', 'partsupp#ps_partkey,ps_suppkey', 'partsupp#ps_partkey,ps_suppkey,ps_availqty', 'partsupp#ps_suppkey', 'partsupp#ps_suppkey,ps_availqty,ps_partkey', 'partsupp#ps_suppkey,ps_partkey', 'partsupp#ps_suppkey,ps_partkey,ps_availqty', 'region#r_name', 'region#r_name,r_regionkey', 'region#r_regionkey,r_name']
len(l)

In [ ]:
#读取../entry/cands.pickle打印成数组，并输出长度
import pickle

def count_distinct_prefix(strings):
    distinct_prefixes = set()
    for string in strings:
        prefix = string.split('#')[0]
        distinct_prefixes.add(prefix)
    return distinct_prefixes

with open("../entry/cands.pickle", "rb") as f:
    cands = pickle.load(f)
    add = list(count_distinct_prefix(cands))
    add_with_prefix = [f"{prefix}#tiflash" for prefix in add]
    print(add_with_prefix)
    cands.extend(add_with_prefix)
    with open('../entry/cands2.pickle', 'wb') as file:
        pickle.dump(cands, file)
    # cands.append("customer#tiflash")
    # print(len(cands))
print(len(cands))
print(cands)




In [ ]:
import pickle

with open("../entry/workload.pickle", "rb") as f:
    workload = pickle.load(f)
    print(workload)

### TPC-DS 的手工候选索引集合

In [ ]:
import pickle



candidate_hand = [
    "store_sales#ss_item_sk,ss_sold_date_sk,ss_ext_sales_price",
    "item#i_item_sk,i_category,i_class,i_item_id,i_item_desc",
    "date_dim#d_date_sk",
    "inventory#inv_item_sk,inv_warehouse_sk,inv_date_sk",
    "warehouse#w_warehouse_sk",
    "item#i_current_price,i_item_sk",
    "date_dim#d_date_sk",
    "customer_address#ca_address_sk",
    "customer#c_current_addr_sk,c_customer_sk",
    "store_sales#ss_customer_sk,ss_sold_date_sk,ss_item_sk",
    "date_dim#d_date_sk,d_month_seq,d_year,d_moy",
    "item#i_item_sk,i_current_price,i_category",
    "web_sales#ws_item_sk,ws_sold_date_sk,ws_ext_sales_price",
    "item#i_item_sk,i_category,i_class,i_item_id,i_item_desc",
    "date_dim#d_date_sk",
    "web_sales#ws_web_page_sk,ws_item_sk,ws_order_number,ws_sold_date_sk,ws_sales_price,ws_net_profit",
    "web_returns#wr_item_sk,wr_order_number,wr_reason_sk,wr_refunded_cdemo_sk,wr_returning_cdemo_sk,wr_refunded_addr_sk,wr_refunded_cash,wr_fee",
    "web_page#wp_web_page_sk",
    "customer_demographics#cd_demo_sk,cd_marital_status,cd_education_status",
    "customer_address#ca_address_sk,ca_country,ca_state",
    "date_dim#d_date_sk,d_year",
    "reason#r_reason_sk",
    "web_sales#ws_item_sk,ws_order_number,ws_sold_date_sk,ws_quantity,ws_net_paid,ws_net_profit",
    "web_returns#wr_order_number,wr_item_sk,wr_return_quantity,wr_return_amt",
    "date_dim#d_date_sk,d_year,d_moy",
    "catalog_sales#cs_order_number,cs_item_sk,cs_quantity,cs_net_paid,cs_net_profit",
    "catalog_returns#cr_order_number,cr_item_sk,cr_return_quantity,cr_return_amount",
    "store_sales#ss_ticket_number,ss_item_sk,ss_quantity,ss_net_paid,ss_net_profit",
    "store_returns#sr_ticket_number,sr_item_sk,sr_return_quantity,sr_return_amt",
    "date_dim#d_date_sk,d_year,d_moy",
    "store_sales#ss_sold_date_sk,ss_item_sk,ss_ext_sales_price",
    "item#i_item_sk,i_brand_id,i_brand,i_manager_id",
    "web_sales#ws_item_sk,ws_sold_date_sk,ws_ext_discount_amt",
    "item#i_manufact_id,i_item_sk",
    "date_dim#d_date,d_date_sk"
]

candidate_hand = list(set(candidate_hand))

# add = ['call_center#tiflash', 'catalog_page#tiflash', 'catalog_returns#tiflash', 'catalog_sales#tiflash', 'customer#tiflash', 'customer_address#tiflash', 'customer_demographics#tiflash', 'date_dim#tiflash', 'dbgen_version#tiflash', 'household_demographics#tiflash', 'income_band#tiflash', 'inventory#tiflash', 'item#tiflash', 'promotion#tiflash', 'reason#tiflash', 'ship_mode#tiflash', 'store#tiflash', 'store_returns#tiflash', 'store_sales#tiflash', 'time_dim#tiflash', 'warehouse#tiflash', 'web_page#tiflash', 'web_returns#tiflash', 'web_sales#tiflash', 'web_site#tiflash']

# candidate_hand.extend(add)

with open("../entry/hand_cands.pickle", "wb") as f:
    pickle.dump(candidate_hand, f)


In [ ]:
file_path = "../workload/tpcds/queries.sql"
queries = []

with open(file_path, 'r') as file:
    query = ""
    for line in file:
        line = line.strip()
        if line.startswith('--') or not line:
            continue  # 跳过注释行和空行
        if line.endswith(';'):
            query += line
            queries.append(query)
            query = ""
        else:
            query += line + " "

# 打印标记后的SQL语句
for i, query in enumerate(queries, start=1):
    print(f"Query {i}: {query}")


with open("../entry/tpcds.pickle", "wb") as f:
    pickle.dump(queries, f)

In [ ]:
a = "hypo_store_sales_ss_item_sk_ss_sold_date_sk_ss_ext_sales_price_idx"

a[:50]

